In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Load the sliding window data
file_path = r'C:\Users\Shadow\Desktop\GIT_X\GIT_PE\thesis\Sliding Window\modi_data.csv'
data = pd.read_csv(file_path)

# Step 1: Identify and clean non-numeric values
# Replace problematic non-numeric values with NaN and convert to float
data = data.apply(pd.to_numeric, errors='coerce')

# Step 2: Drop rows with NaN values
data = data.dropna()

# Preprocess the data
scaler = StandardScaler()
X = scaler.fit_transform(data.iloc[:, :-1])  # Features
y = scaler.fit_transform(data.iloc[:, -1].values.reshape(-1, 1))  # Target

# Train-Test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# Create DataLoader for batching
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64)

# Define the N-Beats Block and Model
class NBeatsBlock(nn.Module):
    def __init__(self, input_size, hidden_size, theta_size):
        super(NBeatsBlock, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, theta_size)  # Only output theta_size (forecasting steps)
        )
        self.backcast_size = input_size
        self.forecast_size = theta_size

    def forward(self, x):
        theta = self.fc(x)
        return theta  # Return only forecast

class NBeatsModel(nn.Module):
    def __init__(self, input_size, hidden_size, theta_size, num_blocks):
        super(NBeatsModel, self).__init__()
        self.blocks = nn.ModuleList([NBeatsBlock(input_size, hidden_size, theta_size) for _ in range(num_blocks)])
    
    def forward(self, x):
        forecast = torch.zeros(x.shape[0], 1).to(x.device)  # Initialize forecast shape [batch_size, 1]
        for block in self.blocks:
            f = block(x)  # No backcast subtraction, only forecast aggregation
            forecast = forecast + f  # Aggregate forecasts
        return forecast

# Model parameters
input_size = X_train.shape[1]  # Number of features
hidden_size = 256  # Hidden layer size
theta_size = 1  # Output size for forecasting (single future value)
num_blocks = 3  # Number of blocks in the model

# Initialize the model
model = NBeatsModel(input_size=input_size, hidden_size=hidden_size, theta_size=theta_size, num_blocks=num_blocks)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    if epoch % 10 == 0:
        print(f'Epoch [{epoch}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}')

# Evaluation
model.eval()
with torch.no_grad():
    test_loss = 0.0
    all_preds = []
    all_targets = []
    for X_batch, y_batch in test_loader:
        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch)
        test_loss += loss.item()
        all_preds.extend(y_pred.cpu().numpy())
        all_targets.extend(y_batch.cpu().numpy())

    print(f'Test Loss (MSE): {test_loss / len(test_loader):.4f}')

# Calculate Evaluation Metrics
all_preds = np.array(all_preds).flatten()
all_targets = np.array(all_targets).flatten()

mae = mean_absolute_error(all_targets, all_preds)
mse = mean_squared_error(all_targets, all_preds)
rmse = np.sqrt(mse)
r2 = r2_score(all_targets, all_preds)

print(f'Mean Absolute Error (MAE): {mae:.4f}')
print(f'Mean Squared Error (MSE): {mse:.4f}')
print(f'Root Mean Squared Error (RMSE): {rmse:.4f}')
print(f'R² (R-squared): {r2:.4f}')

# Make predictions and save to CSV
with torch.no_grad():
    future_forecast = model(X_test_tensor)
    future_forecast = scaler.inverse_transform(future_forecast.numpy())  # Inverse scale

    # Convert to DataFrame
    forecast_df = pd.DataFrame(future_forecast, columns=["Forecast"])

    # Save to CSV
    forecast_df.to_csv('future_forecast.csv', index=False)

    print(f"Predictions saved to 'future_forecast.csv'")


In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Load the sliding window data
file_path = r'C:\Users\Shadow\Desktop\GIT_X\GIT_PE\thesis\Sliding Window\modi_data.csv'
data = pd.read_csv(file_path)

# Step 1: Identify and clean non-numeric values
# Replace problematic non-numeric values with NaN and convert to float
data = data.apply(pd.to_numeric, errors='coerce')

# Step 2: Drop rows with NaN values
data = data.dropna()

# Preprocess the data
scaler = StandardScaler()
X = scaler.fit_transform(data.iloc[:, :-1])  # Features
y = scaler.fit_transform(data.iloc[:, -1].values.reshape(-1, 1))  # Target

# Train-Test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# Create DataLoader for batching
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64)

# Define the N-Beats Block and Model
class NBeatsBlock(nn.Module):
    def __init__(self, input_size, hidden_size, theta_size):
        super(NBeatsBlock, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, theta_size)  # Only output theta_size (forecasting steps)
        )
        self.backcast_size = input_size
        self.forecast_size = theta_size

    def forward(self, x):
        theta = self.fc(x)
        return theta  # Return only forecast

class NBeatsModel(nn.Module):
    def __init__(self, input_size, hidden_size, theta_size, num_blocks):
        super(NBeatsModel, self).__init__()
        self.blocks = nn.ModuleList([NBeatsBlock(input_size, hidden_size, theta_size) for _ in range(num_blocks)])
    
    def forward(self, x):
        forecast = torch.zeros(x.shape[0], 1).to(x.device)  # Initialize forecast shape [batch_size, 1]
        for block in self.blocks:
            f = block(x)  # No backcast subtraction, only forecast aggregation
            forecast = forecast + f  # Aggregate forecasts
        return forecast

# Model parameters
input_size = X_train.shape[1]  # Number of features
hidden_size = 256  # Hidden layer size
theta_size = 1  # Output size for forecasting (single future value)
num_blocks = 3  # Number of blocks in the model

# Initialize the model
model = NBeatsModel(input_size=input_size, hidden_size=hidden_size, theta_size=theta_size, num_blocks=num_blocks)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    if epoch % 10 == 0:
        print(f'Epoch [{epoch}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}')

# Evaluation
model.eval()
with torch.no_grad():
    test_loss = 0.0
    all_preds = []  
    all_targets = []  
    for X_batch, y_batch in test_loader:
        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch)
        test_loss += loss.item()
        all_preds.extend(y_pred.cpu().numpy())
        all_targets.extend(y_batch.cpu().numpy())

    print(f'Test Loss (MSE): {test_loss / len(test_loader):.4f}')

# Calculate Evaluation Metrics
all_preds = np.array(all_preds).flatten()
all_targets = np.array(all_targets).flatten()

mae = mean_absolute_error(all_targets, all_preds)
mse = mean_squared_error(all_targets, all_preds)
rmse = np.sqrt(mse)
r2 = r2_score(all_targets, all_preds)

print(f'Mean Absolute Error (MAE): {mae:.4f}')
print(f'Mean Squared Error (MSE): {mse:.4f}')
print(f'Root Mean Squared Error (RMSE): {rmse:.4f}')
print(f'R² (R-squared): {r2:.4f}')

# Make predictions and save to CSV
with torch.no_grad():
    future_forecast = model(X_test_tensor)
    future_forecast = scaler.inverse_transform(future_forecast.numpy())  # Inverse scale

    # Convert to DataFrame
    forecast_df = pd.DataFrame(future_forecast, columns=["Forecast"])

    # Save to CSV
    forecast_df.to_csv('future_forecast.csv', index=False)

    print(f"Predictions saved to 'future_forecast.csv'")
